
# this is Jon's attempt to put Karl's `alignPupilFMC.py` into an jupyter notebook   

### import stuff

In [1]:
!pip install scipy
!pip install pandas
!pip install numpy
!pip install pyqt5

In [2]:

import numpy as np
import pandas as pd
import json
from pathlib import Path

from scipy import optimize
from scipy.spatial.transform import Rotation 

In [3]:
debug = True
if debug:
    %matplotlib qt
    # %matplotlib 
    import matplotlib.pyplot as plt


### get yr paths right

In [4]:
session_id = 'sesh_2022-02-15_11_54_28_pupil_maybe'
freemocap_data_folder = Path('C:/Users/jonma/Dropbox/FreeMoCapProject/FreeMocap_Data')

session_folder_path = freemocap_data_folder / session_id

mediapipe_skeleton_data_path = session_folder_path / 'DataArrays'/ 'mediaPipeSkel_3d_smoothed.npy'
mediapipe_head_rotations_path = session_folder_path / 'DataArrays'/ 'mediaPipeSkel_3d_head_rotation_matricies_fr_row_col.npy'

pupil_data_path = session_folder_path / 'pupil_000'
pupil_data_exports_path = pupil_data_path / 'exports' / '000'
pupil_positions_path = pupil_data_exports_path / 'pupil_positions.csv'
pupil_recording_info_path = pupil_data_path / 'info.player.json'

freemocap_unix_timestamp_path = session_folder_path /'unix_synced_timestamps.csv'


 ### load in data

- full body 3d kinematic data - `mediapipe_skeleton_frame_mar_xyz`
    - type: numpy array
    - dimensions: [frame_number, tracked_point_number, dimension (xyz)]
- pupil data from `pupil_positions.csv` in the **pupil exports** folder
    - definitions in `pupil_gaze_positions_info.txt`
- pupil start time from `info.player.json` for sync


In [5]:
mediapipe_skeleton_fr_mar_xyz = np.load(mediapipe_skeleton_data_path)
mediapipe_head_rotation_matricies_fr_row_col = np.load(mediapipe_head_rotations_path)
pupil_dataframe = pd.read_csv(pupil_positions_path)
pupil_recording_info_json = json.load(open(pupil_recording_info_path))
freemocap_unix_timestamps_dataframe = pd.read_csv(freemocap_unix_timestamp_path)

### pull out relevant pupil data

data for both eyes

In [6]:

timestamps = np.array(pupil_dataframe['pupil_timestamp'])
eye_theta = np.array(pupil_dataframe['theta'])
eye_phi = np.array(pupil_dataframe['phi'])
sphere_center_x = np.array(pupil_dataframe['sphere_center_x'])
sphere_center_y = np.array(pupil_dataframe['sphere_center_y'])
sphere_center_z = np.array(pupil_dataframe['sphere_center_z'])
pupil_center_normal_x = np.array(pupil_dataframe['circle_3d_normal_x'])
pupil_center_normal_y = np.array(pupil_dataframe['circle_3d_normal_y'])
pupil_center_normal_z = np.array(pupil_dataframe['circle_3d_normal_z'])

eye_d = pupil_dataframe['eye_id']
method = pupil_dataframe['method']
# tracking_method_name = 'pye3d 0.3.0 real-time'




get data for the right eye

pull out data according to `eye_d` (right_eye == 0) and tracking method (because pupil interleaves 2d and 3d data)

In [7]:
right_eye_d = 0 

#timestamps
r_eye_timestamps_og = timestamps[np.logical_and(eye_d==right_eye_d, method!='2d c++')]

r_eye_theta = eye_theta[np.logical_and(eye_d==right_eye_d, method!='2d c++')]
r_eye_phi = eye_phi[np.logical_and(eye_d==right_eye_d, method!='2d c++')]

r_eye_sphere_center_x = sphere_center_x[np.logical_and(eye_d==right_eye_d, method!='2d c++')]
r_eye_sphere_center_y = sphere_center_y[np.logical_and(eye_d==right_eye_d, method!='2d c++')]
r_eye_sphere_center_z = sphere_center_z[np.logical_and(eye_d==right_eye_d, method!='2d c++')]

r_eye_pupil_center_normal_x = pupil_center_normal_x[np.logical_and(eye_d==right_eye_d, method!='2d c++')]
r_eye_pupil_center_normal_y = pupil_center_normal_y[np.logical_and(eye_d==right_eye_d, method!='2d c++')]
r_eye_pupil_center_normal_z = pupil_center_normal_z[np.logical_and(eye_d==right_eye_d, method!='2d c++')]


Get data for the left eye

pull out data according to `eye_d` (left eye == 1) and tracking method (because pupil interleaves 2d and 3d data)

In [8]:
left_eye_d = 1 

#timestamps
l_eye_timestamps_og = timestamps[np.logical_and(eye_d==left_eye_d, method!='2d c++')]

l_eye_theta = eye_theta[np.logical_and(eye_d==left_eye_d, method!='2d c++')]
l_eye_phi = eye_phi[np.logical_and(eye_d==left_eye_d, method!='2d c++')]

l_eye_sphere_center_x = sphere_center_x[np.logical_and(eye_d==left_eye_d, method!='2d c++')]
l_eye_sphere_center_y = sphere_center_y[np.logical_and(eye_d==left_eye_d, method!='2d c++')]
l_eye_sphere_center_z = sphere_center_z[np.logical_and(eye_d==left_eye_d, method!='2d c++')]

l_eye_pupil_center_normal_x = pupil_center_normal_x[np.logical_and(eye_d==left_eye_d, method!='2d c++')]
l_eye_pupil_center_normal_y = pupil_center_normal_y[np.logical_and(eye_d==left_eye_d, method!='2d c++')]
l_eye_pupil_center_normal_z = pupil_center_normal_z[np.logical_and(eye_d==left_eye_d, method!='2d c++')]


In [9]:
if debug:
    fig = plt.figure(num=2134,figsize=(10,10))
    
    ax1 = fig.add_subplot(321)
    ax1.plot(r_eye_timestamps_og, r_eye_theta, '.-', label='r eye theta')
    ax1.plot(r_eye_timestamps_og, r_eye_phi, '.-', label='r eye phi')

    ax1.legend(loc='upper right')
    
    ax3 = fig.add_subplot(323)
    ax3.plot(r_eye_timestamps_og, r_eye_pupil_center_normal_x, '.-', label='r_eye_pupil_center_normal_x')
    ax3.plot(r_eye_timestamps_og, r_eye_pupil_center_normal_y, '.-', label='r_eye_pupil_center_normal_y')
    ax3.plot(r_eye_timestamps_og, r_eye_pupil_center_normal_z, '.-', label='r_eye_pupil_center_normal_z')
    ax3.legend(loc='upper right')

    ax5 = fig.add_subplot(325)
    ax5.plot(r_eye_timestamps_og, r_eye_sphere_center_x, '.-', label='r_eye_sphere_center_x')
    ax5.plot(r_eye_timestamps_og, r_eye_sphere_center_y, '.-', label='r_eye_sphere_center_y')
    ax5.plot(r_eye_timestamps_og, r_eye_sphere_center_z, '.-', label='r_eye_sphere_center_z')
    ax5.legend(loc='upper right')

    
    ax2 = fig.add_subplot(322)
    ax2.plot(l_eye_timestamps_og, l_eye_theta, '.-', label='l eye theta')
    ax2.plot(l_eye_timestamps_og, l_eye_phi, '.-', label='l eye phi')
    ax2.legend(loc='upper right')
    
    ax4 = fig.add_subplot(324)
    ax4.plot(l_eye_timestamps_og, l_eye_pupil_center_normal_x, '.-', label='l_eye_pupil_center_normal_x')
    ax4.plot(l_eye_timestamps_og, l_eye_pupil_center_normal_y, '.-', label='l_eye_pupil_center_normal_y')
    ax4.plot(l_eye_timestamps_og, l_eye_pupil_center_normal_z, '.-', label='l_eye_pupil_center_normal_z')
    ax4.legend(loc='upper right')

    ax6 = fig.add_subplot(326)
    ax6.plot(l_eye_timestamps_og, l_eye_sphere_center_x, '.-', label='l_eye_sphere_center_x')
    ax6.plot(l_eye_timestamps_og, l_eye_sphere_center_y, '.-', label='l_eye_sphere_center_y')
    ax6.plot(l_eye_timestamps_og, l_eye_sphere_center_z, '.-', label='l_eye_sphere_center_z')

    plt.pause(.1)
    plt.show()


### create XYZ gaze array


subtract eyeball center from pupil center to re-base the gaze vector to the origin  

skipping 'remove nans' step for now, but i might need to add it later? 

### synchronize pupil and freemocap timestamps 

#### convert pupil timestamps into unix time

In [10]:
r_eye_timestamps =  r_eye_timestamps_og - pupil_recording_info_json['start_time_synced_s'] +pupil_recording_info_json['start_time_system_s'] 
l_eye_timestamps =  l_eye_timestamps_og - pupil_recording_info_json['start_time_synced_s'] +pupil_recording_info_json['start_time_system_s'] 



calculate freemocap unix timestamps (as mean of each camera's timestamp on each frame)

In [11]:

each_cam_timestamps_list = []

camera_names = [column_name for column_name in list(freemocap_unix_timestamps_dataframe.columns) if 'Cam' in column_name]

for this_camera_name in camera_names:
    each_cam_timestamps_list.append(np.array(freemocap_unix_timestamps_dataframe[this_camera_name]))
    
cameras_timestamp_array = np.stack(each_cam_timestamps_list,axis=1)
cameras_timestamp_array[cameras_timestamp_array==-1]=np.nan #replace -1 with nan

freemocap_timestamps = np.nanmean(cameras_timestamp_array,axis=1)

freemocap_frame_durations = np.diff(freemocap_timestamps)
freemocap_frames_per_second = np.nanmean(freemocap_frame_durations**-1)


In [12]:
if debug:

    timestamp_diff_on_each_frame_rel_cam0 = np.diff(cameras_timestamp_array, axis=1)
    frame_duration = np.diff(freemocap_timestamps)

    fig = plt.figure(234899234)
    ax1 =  fig.add_subplot(411)
    ax1.plot(cameras_timestamp_array, '.-',label='each cam')
    ax1.plot(freemocap_timestamps, 'r-',label='freemocap_timestamps')
    ax1.set_title('cameras_timestamps')
    ax1.set_xlabel('frame number')
    ax1.set_ylabel('timestamp')
    ax1.legend(loc='upper left')
    
    ax2 =  fig.add_subplot(412)
    for this_cam_num in range(timestamp_diff_on_each_frame_rel_cam0.shape[1]):
        ax2.hist(timestamp_diff_on_each_frame_rel_cam0[:,this_cam_num], label=f'camera {this_cam_num}', alpha=0.5)
    ax2.set_xlim(-.1,.1)
    ax2.set_xlabel('timestamp diff on each frame (s)')
    ax2.legend()

    ax3 =  fig.add_subplot(413)
    ax3.plot(freemocap_frame_durations, '.-',label='frame duration')
    ax3.plot([0, len(freemocap_frame_durations)],
            [np.nanmean(freemocap_frame_durations), np.nanmean(freemocap_frame_durations)],
            'r-',
            label=f'mean frame duration (ms):{str(np.nanmean(freemocap_frame_durations)*1000)[:5]}',
            )
    ax3.legend(loc='upper left')
    ax3.set_xlabel('frame number')
    ax3.set_ylabel('frame duration (s)')

    ax4 =  fig.add_subplot(414)
    ax4.hist(freemocap_frame_durations**-1, label='frame duration')
    ax4.legend(loc='upper left')
    ax4.set_xlabel('frame rate (frames per second')
    ax4.set_xlim(0,50)
    
    plt.pause(.1)
    plt.show()

In [13]:
print(f'freemocap_timestamps[-1]: {freemocap_timestamps[-1]}')
print(f'r_eye_timestamps[-1]: {r_eye_timestamps[-1]}')

np.min((freemocap_timestamps[-1],r_eye_timestamps[-1]))

freemocap_timestamps[-1]: 1644944570.8511524
r_eye_timestamps[-1]: 1644944575.0656984


1644944570.8511524

#### align freemocap and pupil timestamps and clip the starts and ends of the various data traces so that everything covers the same timespacn

In [14]:

#find start and end frames shared by all datastreams
start_time_unix = np.max((freemocap_timestamps[0],r_eye_timestamps[0], l_eye_timestamps[0]))
end_time_unix = np.min((freemocap_timestamps[-1],r_eye_timestamps[-1], l_eye_timestamps[-1]))

#freemocap
if any(freemocap_timestamps>=start_time_unix):
    freemocap_start = np.where(freemocap_timestamps>=start_time_unix)[0][0]
else:
    freemocap_start = 0

if any(freemocap_timestamps<=end_time_unix):
    freemocap_end = np.where(freemocap_timestamps<=end_time_unix)[0][-1]
else:
    freemocap_end = len(freemocap_timestamps)

#right eye
if any(r_eye_timestamps>=start_time_unix):
    r_eye_start = np.where(r_eye_timestamps>=start_time_unix)[0][0]
else:
    r_eye_start = 0
    
if any(r_eye_timestamps<=end_time_unix):
    r_eye_end = np.where(r_eye_timestamps<=end_time_unix)[0][-1]
else:
    r_eye_end = len(r_eye_timestamps)

#left eye
if any(l_eye_timestamps>=start_time_unix):
    l_eye_start = np.where(l_eye_timestamps>=start_time_unix)[0][0]
else:
    l_eye_start = 0
    
if any(l_eye_timestamps<=end_time_unix):
    l_eye_end = np.where(l_eye_timestamps<=end_time_unix)[0][-1]
else:
    l_eye_end = len(l_eye_timestamps)

#rebase time onto freemocap's framerate (b/c it's slower than pupil) <- sloppy, assumes mocap slower than eye tracker, which is untrue for say GoPros
new_timestamps = np.arange(start_time_unix,end_time_unix,1/freemocap_frames_per_second) 


freemocap_timestamps = freemocap_timestamps[freemocap_start:freemocap_end]
r_eye_timestamps = r_eye_timestamps[r_eye_start:r_eye_end]
l_eye_timestamps = l_eye_timestamps[l_eye_start:l_eye_end]



clip out portions of each eye's data stream that correspond to the shared time period

(I appear to be skiping the freemocap data, which might cause fail on other recordings?)

In [15]:
r_eye_sphere_center_x = r_eye_sphere_center_x[r_eye_start:r_eye_end]
r_eye_sphere_center_y = r_eye_sphere_center_y[r_eye_start:r_eye_end]
r_eye_sphere_center_z = r_eye_sphere_center_z[r_eye_start:r_eye_end]

r_eye_pupil_center_normal_x = r_eye_pupil_center_normal_x[r_eye_start:r_eye_end]
r_eye_pupil_center_normal_y = r_eye_pupil_center_normal_y[r_eye_start:r_eye_end]
r_eye_pupil_center_normal_z = r_eye_pupil_center_normal_z[r_eye_start:r_eye_end]

In [16]:
l_eye_sphere_center_x = l_eye_sphere_center_x[l_eye_start:l_eye_end]
l_eye_sphere_center_y = l_eye_sphere_center_y[l_eye_start:l_eye_end]
l_eye_sphere_center_z = l_eye_sphere_center_z[l_eye_start:l_eye_end]

l_eye_pupil_center_normal_x = l_eye_pupil_center_normal_x[l_eye_start:l_eye_end]
l_eye_pupil_center_normal_y = l_eye_pupil_center_normal_y[l_eye_start:l_eye_end]
l_eye_pupil_center_normal_z = l_eye_pupil_center_normal_z[l_eye_start:l_eye_end]

create r_eye_gaze_xyz : the direction the eye is pointing *in head centered coordinates*

In [17]:
r_eye_gaze_x = r_eye_pupil_center_normal_x
r_eye_gaze_y = r_eye_pupil_center_normal_y
r_eye_gaze_z = r_eye_pupil_center_normal_z

r_eye_gaze_xyz_og = np.vstack((r_eye_gaze_x, r_eye_gaze_y, r_eye_gaze_z)).T

print(f'r_eye_gaze_xyz_og.shape: {r_eye_gaze_xyz_og.shape}')

r_eye_gaze_xyz_og.shape: (17737, 3)


In [18]:
l_eye_gaze_x = l_eye_pupil_center_normal_x
l_eye_gaze_y = l_eye_pupil_center_normal_y
l_eye_gaze_z = l_eye_pupil_center_normal_z

l_eye_gaze_xyz_og = np.vstack((l_eye_gaze_x, l_eye_gaze_y, l_eye_gaze_z)).T

print(f'l_eye_gaze_xyz_og.shape: {l_eye_gaze_xyz_og.shape}')

l_eye_gaze_xyz_og.shape: (16324, 3)


#### resample gaze and skeleton data so that each has the same number of frames

resample pupil data onto freemocap timestamps

In [19]:

r_eye_gaze_xyz = np.zeros((len(freemocap_timestamps),3))
l_eye_gaze_xyz = np.zeros((len(freemocap_timestamps),3))

for this_dimension in range(3): #loop over the X Y and Z simensions and resample to new timestamps
    r_eye_gaze_xyz[:,this_dimension] = np.interp(freemocap_timestamps,
                                        r_eye_timestamps,
                                        r_eye_gaze_xyz_og[:,this_dimension])
    l_eye_gaze_xyz[:,this_dimension] = np.interp(freemocap_timestamps,
                                        l_eye_timestamps,
                                        l_eye_gaze_xyz_og[:,this_dimension])


In [20]:
r_eye_gaze_xyz = r_eye_gaze_xyz/np.linalg.norm(r_eye_gaze_xyz,axis=1,ord=2)[:,None]
l_eye_gaze_xyz = l_eye_gaze_xyz/np.linalg.norm(l_eye_gaze_xyz,axis=1,ord=2)[:,None]


# snip off the first frame of the freemocap data becuase there is a -> BUG <- such that there are one fewer timestamp than there are recorded frames. 

V V V V V V

In [21]:
if r_eye_gaze_xyz.shape[0] != mediapipe_skeleton_fr_mar_xyz.shape[0]:
    mediapipe_skeleton_fr_mar_xyz = np.delete(mediapipe_skeleton_fr_mar_xyz, 0, axis=0)
    mediapipe_head_rotation_matricies_fr_row_col = np.delete(mediapipe_head_rotation_matricies_fr_row_col, 0, axis=0)
# DOESN'T BOTHER ME AT ALL HAHA

These should all have the exact same number of frame

In [22]:
print(f'r_eye_gaze_xyz.shape: {r_eye_gaze_xyz.shape}')
print(f'l_eye_gaze_xyz.shape: {l_eye_gaze_xyz.shape}')
print(f'mediapipe_skeleton_fr_mar_xyz.shape: {mediapipe_skeleton_fr_mar_xyz.shape}')
print(f'mediapipe_head_rotation_matricies_fr_row_col.shape: {mediapipe_head_rotation_matricies_fr_row_col.shape}')

r_eye_gaze_xyz.shape: (4915, 3)
l_eye_gaze_xyz.shape: (4915, 3)
mediapipe_skeleton_fr_mar_xyz.shape: (4915, 543, 3)
mediapipe_head_rotation_matricies_fr_row_col.shape: (4915, 3, 3)


In [23]:
if debug:
    fig = plt.figure(num=65341,figsize=(10,20))
    ax1 = fig.add_subplot(211)
    ax1.plot(freemocap_timestamps, r_eye_gaze_xyz[:,0], '.-', label='r_eye_gaze_x')
    ax1.plot(freemocap_timestamps, r_eye_gaze_xyz[:,1], '.-', label='r_eye_gaze_y')
    ax1.plot(freemocap_timestamps, r_eye_gaze_xyz[:,2], '.-', label='r_eye_gaze_z')
    ax1.legend(loc='upper left')
    
    ax2 = fig.add_subplot(212)
    ax2.plot(freemocap_timestamps, l_eye_gaze_xyz[:,0], '.-', label='_eye_gaze_x')
    ax2.plot(freemocap_timestamps, l_eye_gaze_xyz[:,1], '.-', label='_eye_gaze_y')
    ax2.plot(freemocap_timestamps, l_eye_gaze_xyz[:,2], '.-', label='_eye_gaze_z')
    ax2.legend(loc='upper left')
    plt.show()



No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


normalize gaze vector to unit length

save out `npy` of gaze_xyz data

Smooth a bit with savitsky golay filter

In [24]:
from scipy.signal import savgol_filter

smoothWinLength = 5
smoothOrder = 3
for dim in range(r_eye_gaze_xyz.shape[1]):
        r_eye_gaze_xyz[:,dim] = savgol_filter(r_eye_gaze_xyz[:,dim], smoothWinLength, smoothOrder)
        l_eye_gaze_xyz[:,dim] = savgol_filter(l_eye_gaze_xyz[:,dim], smoothWinLength, smoothOrder)


# #reorient gaze (I think because it comes in in image coordiates? not really sure )
# r_eye_gaze_xyz[:,1] = -r_eye_gaze_xyz[:,1]
# r_eye_gaze_xyz[:,(0,2)] = r_eye_gaze_xyz[:,(2,0)]

# l_eye_gaze_xyz[:,1] = -l_eye_gaze_xyz[:,1]
# l_eye_gaze_xyz[:,(0,2)] = l_eye_gaze_xyz[:,(2,0)]

In [25]:
r_gaze_xyz_npy_save_path = pupil_data_exports_path / 'right_eye_gaze_xyz_synchronized_w_freemocap_uncalibrated.npy'
np.save(r_gaze_xyz_npy_save_path, r_eye_gaze_xyz)

l_gaze_xyz_npy_save_path = pupil_data_exports_path / 'left_eye_gaze_xyz_synchronized_w_freemocap_uncalibrated.npy'
np.save(l_gaze_xyz_npy_save_path, l_eye_gaze_xyz)


# VOR Calibration

# prep data for VOR calibration

we'll be finding a rotational offset such that gaze_xyz rotated-by-the offset-then-rotated-by-head-orientation will align with the fixation point

 so we want to put the fixation point (in this case the right index finger in a eyeball-centered reference frame (i.e. where `r_eye_xyz` is at `(0,0,0)`)

In [26]:
vor_frame_start =1000
vor_frame_end = 1400

right_index_fingertip_idx = 41 # pretty sure this is right?
r_eye_idx = 5

fixation_point_xyz = np.squeeze(mediapipe_skeleton_fr_mar_xyz[:, right_index_fingertip_idx, :])
r_eyeball_center_xyz = np.squeeze(mediapipe_skeleton_fr_mar_xyz[:, r_eye_idx, :])
#make sure everyone is the right size and shape
print(f'head_rotation_martices_fr_row_col.shape: {mediapipe_head_rotation_matricies_fr_row_col.shape}')
print(f'r_eye_gaze_xyz.shape: {r_eye_gaze_xyz.shape}')
print(f'fixation_point_xyz.shape: {fixation_point_xyz.shape}')


head_rotation_martices_fr_row_col.shape: (4915, 3, 3)
r_eye_gaze_xyz.shape: (4915, 3)
fixation_point_xyz.shape: (4915, 3)


In [27]:

fixation_point_during_vor_xyz = mediapipe_skeleton_fr_mar_xyz[vor_frame_start:vor_frame_end, right_index_fingertip_idx, :]
r_eyeball_center_during_vor_xyz = mediapipe_skeleton_fr_mar_xyz[vor_frame_start:vor_frame_end, r_eye_idx, :]

fixation_point_in_r_eyeball_coordinates_fr_xyz = r_eyeball_center_during_vor_xyz - fixation_point_during_vor_xyz #Subtract the-thing-that-will-be-at-origin from the-other-things to put other-things in ## thing-that-will-be-at-origin-centered-reference-frame
fixation_point_distance_xyz = np.squeeze(np.linalg.norm(fixation_point_in_r_eyeball_coordinates_fr_xyz,axis=1,ord=2)[:,None])



r_gaze_during_vor_fr_xyz = r_eye_gaze_xyz[vor_frame_start:vor_frame_end]
r_gaze_during_vor_fr_xyz[:,0] = r_gaze_during_vor_fr_xyz[:,0] * fixation_point_distance_xyz #scale gaze vector to reach fixation point
r_gaze_during_vor_fr_xyz[:,1] = r_gaze_during_vor_fr_xyz[:,1] * fixation_point_distance_xyz #scale gaze vector to reach fixation point
r_gaze_during_vor_fr_xyz[:,2] = r_gaze_during_vor_fr_xyz[:,2] * fixation_point_distance_xyz #scale gaze vector to reach fixation point
r_gaze_during_vor_fr_xyz

array([[-140.99623459,  371.93667349, -908.18373013],
       [-162.83809499,  344.26373174, -912.30887986],
       [-165.73504142,  346.97592809, -911.18040645],
       ...,
       [-104.44826484,  -31.40865991, -522.0636855 ],
       [-101.70694275,  -47.25579649, -518.36201151],
       [-102.06426831,  -55.07294944, -524.55510472]])

In [28]:

head_rotation_matrices_during_vor_fr_row_col = mediapipe_head_rotation_matricies_fr_row_col[vor_frame_start:vor_frame_end, :, :]

#make sure everyone is the right size and shape
print(f'head_rotation_matrices_during_vor_fr_row_col.shape: {head_rotation_matrices_during_vor_fr_row_col.shape}')
print(f'r_gaze_during_vor_xyz.shape: {r_gaze_during_vor_fr_xyz.shape}')
print(f'fixation_point_in_r_eyeball_coordinates_xyz.shape: {fixation_point_in_r_eyeball_coordinates_fr_xyz.shape}')


head_rotation_matrices_during_vor_fr_row_col.shape: (400, 3, 3)
r_gaze_during_vor_xyz.shape: (400, 3)
fixation_point_in_r_eyeball_coordinates_xyz.shape: (400, 3)


put mediapipe skel into eyeball centered coordinates (for debugging)

In [29]:
skel_during_vor_fr_mar_dim = mediapipe_skeleton_fr_mar_xyz[vor_frame_start:vor_frame_end,:,:]

for this_marker_number in range(skel_during_vor_fr_mar_dim.shape[1]):
    skel_during_vor_fr_mar_dim[:,this_marker_number,:] = r_eyeball_center_during_vor_xyz - skel_during_vor_fr_mar_dim[:,this_marker_number,:]


print(f'skel_during_vor_fr_mar_dim.shape: {skel_during_vor_fr_mar_dim.shape}')

skel_during_vor_fr_mar_dim.shape: (400, 543, 3)


# define error functions and run optimization

according to  lines  58:68 of - [https://github.com/trentwirth/ARGP_Matlab_LaserSkeleton/blob/main/Toolbox/vorPupilAlignErrFun_eyeCam.m](https://github.com/trentwirth/ARGP_Matlab_LaserSkeleton/blob/main/Toolbox/vorPupilAlignErrFun_eyeCam.m)
```

%%
%%%%%%% This part's important - Rotate gaze vector by [this guess at the proper alignment matrix], prior to resituating  the origin on on the eyeball
%%%%


for rr = 1:length(gazeXYZ)
    
    thisET_frame_unrot = camRotMatGuess * [gazeXYZ(rr,1); gazeXYZ(rr,2); gazeXYZ(rr,3)];
    thisETframe = headRotMat_row_col_fr(:,:,rr) * thisET_frame_unrot;
    
    headOrVec(rr,:) =     headRotMat_row_col_fr(:,:,rr) * [2e3; 0;0];
    
    gazeXYZ(rr,:) = thisETframe;
    
end

```

we want to rotate by rotation_guess THEN by head rotation 

(which makes sense, since we're using the optimizer to find the rotational offset between the eye camera reference frame and the eye-in-head reference frame)



### optimization error animation function 


In [30]:
plt.close('all')


In [32]:

if debug:
    def plot_optimization_error(error,gaze_xyz, gaze_rotated_by_guess_then_head_rotation_xyz, fixation_point_in_eye_coordinates_xyz, skel_during_vor_fr_mar_dim):
        figure_number=13451

        if not plt.fignum_exists(figure_number):
            fig = plt.figure(figure_number)
            ax = fig.add_subplot(111, projection='3d')
        else:
            fig = plt.gcf()
            ax = plt.gca()


        fig.suptitle(f'error: {error}')
        ax_range = 1e3
        ax.clear() 

        ax.plot(0,0,0, 'mo',label='origin')

        ax.plot(gaze_xyz[:,0],
                gaze_xyz[:,1], 
                gaze_xyz[:,2],  'k-o',label='original gaze xyz')

        gaze_rot_xyz  =np.array(gaze_rotated_by_guess_then_head_rotation_xyz)
        ax.plot(gaze_rot_xyz[:,0],
                gaze_rot_xyz[:,1], 
                gaze_rot_xyz[:,2],  'r-o',label='gaze_rotated_by_guess_then_head_rotation_xyz')

        ax.plot(fixation_point_in_eye_coordinates_xyz[:,0],
                fixation_point_in_eye_coordinates_xyz[:,1], 
                fixation_point_in_eye_coordinates_xyz[:,2],  'b-o',label='fixation_point_in_eye_coordinates_xyz')

        ax.set_xlim([-ax_range, ax_range])
        ax.set_ylim([-ax_range, ax_range])
        ax.set_zlim([-ax_range, ax_range])
        ax.legend()
        plt.pause(.01)



def get_error_between_two_rotation_matricies(euler_angle_guess,
                                             gaze_xyz,
                                             fixation_point_in_eye_coordinates_xyz,
                                             head_rotation_maticies_fr_row_col,
                                             skel_during_vor_fr_mar_dim):
    #convert euler angles to rotation matrix
    rotation_matrix_guess = Rotation.from_euler('XYZ',euler_angle_guess).as_matrix()

    #rotate gaze by rotation guess
    gaze_rotated_by_guess = [rotation_matrix_guess @ gaze_xyz[this_frame_number,:] for this_frame_number in range(gaze_xyz.shape[0])]

    #...then rotate THAT by head_rotation matrix
    gaze_rotated_by_guess_then_head_rotation_xyz =[head_rotation_maticies_fr_row_col[this_frame_number,:,:] @ gaze_rotated_by_guess[this_frame_number]
                                                     for this_frame_number in range(gaze_xyz.shape[0])]

    # mean_fixation_point_xyz = np.mean(fixation_point_in_eye_coordinates_xyz, axis=0)
    #define error as difference between the fixation point and that rotated gaze estimate (these are both normalized, I think)
    error_per_frame = gaze_rotated_by_guess_then_head_rotation_xyz - fixation_point_in_eye_coordinates_xyz
    error = np.nanmean(np.nansum(error_per_frame**2)/error_per_frame.shape[0])
    
    plot_optimization_error(error,gaze_xyz, gaze_rotated_by_guess_then_head_rotation_xyz, fixation_point_in_eye_coordinates_xyz, skel_during_vor_fr_mar_dim)

    return error


def get_optimal_rotation_matrix_to_align_gaze_with_target(gaze_xyz,
                                                          fixation_point_in_eye_coordinates_xyz,
                                                          head_rotation_matricies_fr_row_col,
                                                          skel_during_vor_fr_mar_dim):
    euler_angles = optimize.least_squares(get_error_between_two_rotation_matricies,
                                    [0,0,0],
                                    args=(gaze_xyz,fixation_point_in_eye_coordinates_xyz, head_rotation_matricies_fr_row_col, skel_during_vor_fr_mar_dim),
                                    gtol=1e-10,
                                    verbose=2).x
    return Rotation.from_euler('XYZ',euler_angles).as_matrix()


######################################


rotation_matrix_to_align_pupil_data_to_eye_in_head_coordinates = get_optimal_rotation_matrix_to_align_gaze_with_target(r_gaze_during_vor_fr_xyz,
                                                                                fixation_point_in_r_eyeball_coordinates_fr_xyz,
                                                                                head_rotation_matrices_during_vor_fr_row_col,
                                                                                skel_during_vor_fr_mar_dim)

r_eye_gaze_xyz_calibrated = [rotation_matrix_to_align_pupil_data_to_eye_in_head_coordinates @ r_eye_gaze_xyz[this_frame_number,:] for this_frame_number in range(r_eye_gaze_xyz.shape[0])]




   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.9160e+11                                    2.28e+11    
       1              2         2.9245e+10      1.62e+11       1.00e+00       6.24e+10    
       2              4         1.0953e+10      1.83e+10       5.00e-01       1.18e+10    
       3              6         9.8908e+09      1.06e+09       2.50e-01       3.05e+09    
       4              8         9.7599e+09      1.31e+08       6.25e-02       4.77e+08    
       5              9         9.7498e+09      1.01e+07       6.25e-02       3.66e+08    
       6             10         9.7457e+09      4.13e+06       1.56e-02       1.41e+08    
       7             12         9.7451e+09      5.70e+05       3.91e-03       1.34e+08    
       8             13         9.7447e+09      4.71e+05       3.91e-03       9.35e+07    
       9             14         9.7438e+09      8.17e+05       7.81e-03       8.51e+07    

In [33]:
gaze_xyz_npy_save_path = pupil_data_exports_path / 'right_eye_gaze_xyz.npy'
np.save(gaze_xyz_npy_save_path, r_eye_gaze_xyz_calibrated)

In [ ]:
import plotly.offline as py
import plotly.express as px 
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(r_eye_gaze_xyz.shape[0]), y=r_eye_gaze_xyz[:,0], mode='lines+markers'))
fig.add_trace(go.Scatter(x=np.arange(r_eye_gaze_xyz.shape[0]), y=r_eye_gaze_xyz[:,1], mode='lines+markers'))
fig.add_trace(go.Scatter(x=np.arange(r_eye_gaze_xyz.shape[0]), y=r_eye_gaze_xyz[:,2], mode='lines+markers'))
fig.show()